# 인코더 토큰화 층
- 텍스트를 적절한 단위로 나누고 숫자로 변환
- 자음,모음과 같은 가장 작은 단위로 하거나 음절, 단어로 토큰화 진행
- 자음,모음의 경우 단어의 의미가 퇴색
- 단어로 토큰화 할 경우 텍스트의 의미가 잘 유지 되지만 사전의 크기가 커짐
- 최근은 ***서브 워드 토큰화*** 방식을 사용
    - 자주 나오는 단어는 단어 단위 그대로 유지 하고 가끔 나오는 단어는 더 작은 단위로 나눔
    - 예를 들어 "대한민국","안녕"과 같은 단어는 유지, 외국어나 특수문자, 이모티콘은 작게 나눔

#### 토큰화

In [1]:
#띄어쓰기 단위 토큰화
input_text="나는 이번에 일본 여행을 다녀왔다."
input_text_list = input_text.split()
print(f"input_text_list :{input_text_list}")

input_text_list :['나는', '이번에', '일본', '여행을', '다녀왔다.']


In [2]:
#토큰, 아이디 딕셔너리와 아이디, 토큰 딕셔너리 만들기
str2idx={word:idx for idx,word in enumerate(input_text_list)}
idx2str={idx:word for idx,word in enumerate(input_text_list)}
print(str2idx)
print(idx2str)
#토큰을 토큰 아이디로 변환
input_ids= [str2idx[word] for word in input_text_list]
print(input_ids)


{'나는': 0, '이번에': 1, '일본': 2, '여행을': 3, '다녀왔다.': 4}
{0: '나는', 1: '이번에', 2: '일본', 3: '여행을', 4: '다녀왔다.'}
[0, 1, 2, 3, 4]


#### 임베딩
- 토큰화는 숫자일 뿐이므로 의미를 담을 수 없기 때문에 데이터를 의미를 담아 숫자 집합으로 변환 하는 **임베딩** 과정 필요
- Pytorch의 nn.Embedding 클래스 사용

In [3]:
import torch
import torch.nn as nn
embedding_dim=16
embed_layer=nn.Embedding(len(str2idx),embedding_dim)

input_embeddings= embed_layer (torch.tensor(input_ids))
#1개의 문장 -> 1개 차원 설정
input_embeddings=input_embeddings.unsqueeze(0)
input_embeddings.shape

torch.Size([1, 5, 16])

#### 위치 인코딩
- RNN은 순차적으로 처리 하지만 Transformer는 모든 입력을 동시에 처리하는데, 그 과정에서 위치 정보가 사라지게 되니 텍스트에서는 위치가 중요한 정보이기 때문에 위치 인코딩을 해주어야 함
- *Attention is All you need* 에서는 sin,cos 을 활용해 위치 정보를 입력했지만 이후 위치 인코딩도 위치에 따른 임베딩층을 추가해 학습 데이터를 통해 학습하는 방식을 많이 활용
- 수식을 통한 위치 정보 추가 방식이나 임베딩으로 위치 정보 학습 방식 모두 결국 추론 수행 시험에서 입력 토큰의 위치에 따라 고정된 임베딩을 더해주기 때문에 <span  style='color:red'>**절대적 위치 인코딩**</span>이라고 함
- 긴 텍스트를 추론하는 경우 성능이 떨어져 최근에는 <span  style='color:blue'> **상대적 위치 인코딩**</span> 을 많이 활용

In [4]:
#absolute position encoding
embedding_dim =16
max_position=12
embed_layer=nn.Embedding(len(str2idx),embedding_dim)
position_embed_layer=nn.Embedding(max_position,embedding_dim)

position_ids=torch.arange(len(input_ids),dtype=torch.long).unsqueeze(0)
position_encodings= position_embed_layer(position_ids)
token_embeddings= embed_layer(torch.tensor(input_ids))
token_embeddings= token_embeddings.unsqueeze(0)
input_embeddings=token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

# 어텐션
- 앞 뒤 단어 없이'배'라는 단어가 있을때 ship인지 peer인지 알수 없음. 어텐션은 '나는 배 타고 다녀왔다' 타고,다녀왔다 에 주의(attention)해 '배'가 ship으로 해석한 것이다.
- 단어 재해석의 어텐션 연산을 만드려면 **단어와 단어 사이의 관계를 계산해서 그 값에 따라 관련이 깊은 단어와 그렇지 않은 단어를 구분 해야함**


### 쿼리(Query),키(Key),값(Value)
- 우리가 입력하는 검색어 <span style="color:blue">**쿼리**</span>
- 쿼리와 관련있는지 계산하기 위해 문서가 가진 특징을 <span style="color:blue">**키**</span>
- 검색 엔진이 쿼리와 관련이 깊은 키를 가진 문서를 찾아 관련도 순으로 정렬해서 문서를 제공 할 때 <span style="color:blue">**값**</span>
- 예시 문장에서  <span style="color:blue">** '배'가 쿼리**</span> , 키는 문장속의 각 단어
- '배' 라는 쿼리로 ['나는','타고','다녀왔다'] 라는 키 묶음 에서 '타고','다녀왔다'가 적절히 섞여 높은 값이 된다면 단어 재해석을 모방할 수있다.

In [5]:
## 쿼리,키,값 벡터 만드는 nn.Linear층
head_dim = 16
weight_q = nn.Linear(embedding_dim,head_dim)
weight_k = nn.Linear(embedding_dim,head_dim)
weight_v = nn.Linear(embedding_dim,head_dim)

querys=weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)


#### 논문에서 처음 사용 된 스케일 점곱 방식 사용하기
1. 쿼리와 키를 곱하고 분산을 줄이기 위해 임베딩 차원수의 제곱근으로 나눔
2. 쿼리와 키를 곱한 scores를 합이 1이 되도록 softmax 취해 가중치로 변환
3. 가중치와 값을 곱해 동일한 형태의 출력 반환

In [6]:
from math import sqrt
import torch.nn.functional as F
def compute_attention(querys,keys,values,is_causal=False):
    dim_k= querys.size(-1)
    scores=querys @ keys.transpose(-2,-1) / sqrt(dim_k) # 곱하기 위한 (5,16) Transpose
    weights=F.softmax(scores,dim=-1)
    return weights @ values

In [7]:
print("원본 값 :",input_embeddings.shape)
after_attention_embeddings = compute_attention(querys,keys,values)
print("어텐션 적용 :",after_attention_embeddings.shape)

원본 값 : torch.Size([1, 5, 16])
어텐션 적용 : torch.Size([1, 5, 16])


In [8]:
#현재 과정 AttentionHead 클래스 만들기
class AttentionHead(nn.Module):
    #쿼리,키,값 벡터 생성
    def __init__(self,token_embed_dim,head_dim,is_causal=False):
        super().__init__()
        self.is_causal=is_causal
        self.weight_q=nn.Linear(token_embed_dim,head_dim)
        self.weight_k=nn.Linear(token_embed_dim,head_dim)
        self.weight_v=nn.Linear(token_embed_dim,head_dim)
    #compute_attention으로 어텐션 연산 수행
    def forward(self,querys,keys,values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_causal=self.is_causal
        )
        return outputs
#embedding
attention_head = AttentionHead(embedding_dim,embedding_dim)
after_attention_embeddings = attention_head(input_embeddings,input_embeddings,input_embeddings)

### 멀티 헤드 어텐션
토큰사이의 관계를 한 가지 측면에서 이해하는 것보다 여러 측면을 동시에 고려할 때 언어나 문장에 대한 이해도가 높아짐
1. AttentionHead와 동일하지만 헤드의 수(n_head)만큼 연산을 수행 하기 위해 쿼리,키,값을 n_head개로 쪼갬
2. 각각의 어텐션 계산
3. 입력과 같은 형태로 다시 변환
4. 마지막으로 선형층을 통과시키고 최종 결과 반환

In [9]:
# 헤드 수 4로 두고 멀티 헤드 어텐션 구현
class MultiheadAttention(nn.Module):
    def __init__(self,token_embed_dim,d_model,n_head,is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        self.weight_q=nn.Linear(token_embed_dim,head_dim)
        self.weight_k=nn.Linear(token_embed_dim,head_dim)
        self.weight_v=nn.Linear(token_embed_dim,head_dim)
        self.concat_linear=nn.Linear(d_model,d_model)
    def forward(self,querys,keys,values):
        B,T,C = querys.size()
        querys=self.weight_q(querys).view(B,T,self.n_head,C // self.n_head).transpose(1,2)
        keys=self.weight_k(keys).view(B,T,self.n_head,C // self.n_head).transpose(1,2)
        values=self.weight_v(values).view(B,T,self.n_head,C // self.n_head).transpose(1,2)
        attention = compute_attention(querys,keys,values,self.is_causal)
        output = attention.transpose(1,2).contiguous().view(B,T,C)
        output = self.concat_linear(output)
        return output
n_head = 4
mh_attention = MultiheadAttention(embedding_dim,embedding_dim,n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape


torch.Size([1, 5, 16])

# 정규화 & 피드 포워드
- 정규화 : 딥러닝 모델에서 <span style='color: yellow'>**input 이 일정한 분포를 갖도록 만들어 학습이 안정적이고 빨라질 수 있도록 하는 기법**</span>
    - 과거에는 batch input data 사이에 정규화를 수행하는 **배치 정규화**를 사용하지만
    - 트랜스포머 아키텍쳐에서는 feature 차원에서 정규화를 수행하는 **층 정규화** 사용


### 층정규화 (Layer normalization)
- 예를 들어 사람의 키는 140cm~200cm사이의 값을 가진다고 하면 mm로 단위로 변환 시 1400mm~2000mm의 단위로 변경된다. 단위 변경 시 데이터의 분포가 훨씬 넓어지고 모델이 데이터를 과대평가 할 가능성이 높아짐. 이러한 문제를 방지하기 위해 정규화를 해 **모든 데이터가 비슷한 range 와 distribution을 갖도록** 조정
- 딥러닝 분야에서는 layer와 layer 사이에 정규화를 추가해 학습을 안정적으로 만드는 기법을 사용
- 벡터 $X$ 를 정규화 한 $norm_x$ 는 벡터 $X$에서 $x$의 평균을 빼고 $x$의 표준편차로 나눠 평균이 0이고 표준 편차가 1인 분포를 갖게 됨
$$norm_x = (X - 평균)/표준편차$$
- 딥러닝 에서는 평균과 표준편차를 구할 데이터를 어떻게 묶는지에 따라 배치 정규화와 층 정규화로 구분
- 배치 정규화의 경우 자연어 처리에서는 입력 문장 데이터 길이가 다양해 정규화 포함 데이터의 수가 제각각이라 정규화 효과 보장 X,
- 층 정규화의 경우 각 토큰 임베딩의 평균과 표준편차를 이용해 정규화를 수행해 데이터의 수가 다르더라도 정규화 효과에 차이가 없음
#### 트랜스포머에서의 **층 정규화**
1. 피드포워드 층 이후 정규화
    - 사후 정규화(post-norm) 라고 함
2. 먼저 층 정규화 이후 어텐션과 피드포워드 층 통과
    - 사전 정규화(pre-norm) 라고 함

>On Layer Normalization in the Trnasformer Architecture 에 따르면 사전 정규화 방식이 학습이 더 안정적
   

In [11]:
print(embedding_dim)
print(input_embeddings)

16
tensor([[[-1.7263,  0.6910, -2.2461, -0.4709, -1.7675, -0.5451, -0.7644,
          -1.9052, -1.3890, -0.7695,  1.5643,  1.4007, -3.1769, -1.3379,
           0.3272, -1.5821],
         [-1.7494,  0.5201,  0.7756, -0.3975, -1.4791, -0.5822, -0.4589,
          -0.6575,  0.1763, -0.1878,  1.7282,  0.6147,  0.4074,  1.8509,
           0.1097,  0.2598],
         [-1.0922, -0.7020, -0.9251, -0.4595, -1.4528, -1.7481, -1.0573,
           0.1915, -2.4274, -1.2042,  0.0666, -0.7353, -0.4523, -0.7124,
           1.8757, -0.9058],
         [ 0.5254, -0.2783, -0.2952, -0.7015,  1.9352, -1.1526, -0.1530,
          -1.7138, -4.4138, -0.3121,  1.6041,  3.2342,  0.8559,  0.4552,
          -2.8547,  0.3139],
         [ 0.1104, -1.2875, -1.9275,  1.2991,  0.3635, -1.9066, -1.4731,
          -1.7893, -0.7840,  1.6650,  0.1627, -1.3894,  1.3903, -0.9919,
           2.6928, -1.3044]]], grad_fn=<AddBackward0>)


In [12]:
# 사전 층 정규화 구현
norm = nn.LayerNorm(embedding_dim)
norm_x=norm(input_embeddings)
norm_x.shape

torch.Size([1, 5, 16])

In [13]:
norm_x.mean(dim=-1).data,norm_x.std(dim=-1)

(tensor([[ 3.7253e-08, -1.8626e-08,  7.4506e-09,  2.2352e-08, -1.4901e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]], grad_fn=<StdBackward0>))


### 피드포워드 층
- 데이터의 특징을 학습하는 **완전 연결 층(fully connected layer)**
- 멀티 헤드 어텐션이 단어사이의 관계 파악 역할이라면 **피드포워드 층은 입력 텍스트 전체를 이해**하는 역할
- **선형 층,드롭아웃 층, 층 정규화,활성 함수로 구성**하고 임베딩의 차원을 동일하게 유지해야 쉽게 층을 쌓아 확장이 가능하기 때문에 **입력과 출력이 동일하도록**
- 일반적으로 d_model 차원에서 d_model보다 2~3배 큰 dim_feedforward 차원으로 확장했다가 다시 d_model로 변환

In [14]:
#피드 포워드 층 구현
class PreLayerNormFeedForward(nn.Module):
    def __init__(self,d_model,dim_feedforward,dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 층 1 
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 층 2
        self.activation = nn.GELU() # 활성 함수
        self.norm = nn.LayerNorm(d_model) #층 정규화
    def forward(self,src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
        x = self.dropout2(x)
        return x


## 인코더 
- 잔차 연결 (residual connection)이 있는 데 안정적으로 학습이 가능하도록 도와줌
- $N_e$번 반복된다고 표시되어 있는데 트랜스포머 인코더는 인코더 블록을 반복해서 쌓아서 만듬

In [15]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super().__init__()
        self.attn = MultiheadAttention(d_model, d_model, nhead) #멀티 헤드 어텐션 클래스
        self.norm1 = nn.LayerNorm(d_model) # 층 정규화
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) #피드 포워드
    def forward(self,src):
        norm_x = self.norm1(src) # 층 정규화 실시
        attn_output = self.attn(norm_x, norm_x, norm_x) # 멀티 헤드 어텐션 연산 수행
        x = src + self.dropout1(attn_output) #  잔차 연결을 위해 어텐션 결과에 드롭 아웃한 값과 입력값 합
        x = self.feed_forward(x) # 피드포워드 연산
        return x

#### $N_e$ 번 반복하게 코드 구현하자

In [17]:
# $N_e$
import copy
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])
class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
    def forward(self,src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return output


## 디코더
1. 인코더는 기본적인 멀티헤드 어텐션을 사용하지만, 디코더에서는 <span style='color:yellow'>**마스크 멀티헤드 어텐션 사용**</span>
2. 디코더는 생성을 담당하는 부분으로, 사람이 글을 쓸때 앞단어 부터 작성하듯이 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성함. (causal, auto-regressive 특징)

#### 디코더 특징
- 실제 텍스트 생성 시 디코더는 이전 까지 생성한 텍스트만 확인 할 수 있다. 그런데 학습할 때 인코더와 디코더 모두 완성된 텍스트를 입력받는다. 따라서 <span style='color : green'>**어텐션을 그대로 사용 시 미래 시점에 작성해야 하는 텍스트를 미리 확인하게 되는 문제가 생김**</span>
- 이를 막기 위해 이전 생성 토큰만 확인 할 수 있도록 마스크 추가. 어텐션 코드에 is_causal을 추가해 디코더의 경우 True로 설정해 마스크 연산 추가
- is_causal 이 참일 때 torch.ones로 모두 1인 행렬에 trill 함수를 취해 대각선 아래 부분만 1로 유지되고 나머지는 음의 무한대로 변경해 마스크 생성

In [18]:
#디코더 어텐션 연산(마스크 추가)
def compute_attention(querys, keys, values, is_causal = False):
    dim_k = querys.size(-1)
    scores = querys @ keys.transpose(-2,-1) / sqrt(dim_k)
    if is_causal:
        query_length = querys.size(2)
        key_length = keys.size(2)
        temp_mask = torch.ones(query_length, key_length, dtype = torch.bool).trill(diagonal=0)
        scores = scores.masked_fill(temp_mask == False, float("-inf"))
    weights = F.softmax(scores, dim= -1)
    return weights @ values

#### 크로스 어텐션
- 예를 들어 영어를 한국어로 번역 할 때 영어 문장을 입력으로 받아 처리한 결과를 번역한 한국어를 생성하는 디코더가 받아서 활용.
- 이때 쿼리는 디코더의 잠재 상태를 사용하고 key 와 value 는 인코더의 결과를 사용한다.

In [19]:
#크로스 어텐션이 포함된 디코더 층
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, d_model, nhead)
        self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    def forward(self, tgt, encoder_output, is_causal = True):
        #셀프 어텐션
        x = self.norm1(tgt)
        x = x + self.dropout1(self.self_attn(x,x,x))
        #크로스 어텐션
        x = self.norm2(x)
        x=  x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output, is_causal = is_causal))
        #피드 포워드
        x = self.feed_forward(x)
        return x

#### $N$번 반복하게 만들기

In [20]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])
class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
    def forward(self,tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(tgt, src)
        return output

<table>
    <thead>
        <tr>
            <th>모델 그룹</th>
            <th>대표</th>
            <th>장점</th>
            <th>단점</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>인코더(NLU)</td>
            <td>BERT</td>
            <td>양방향 이해를 통해 자연어 이해에서 디코더 모델 대비 높은 성능</td>
            <td>자연어 생성 작업(NLG)에 부적합 하고 컨텍스트의 길이가 제한적</td>
        </tr>
        <tr>
            <td>디코더(NLG)</td>
            <td>GPT</td>
            <td>생성작업에서 뛰어난 성능,긴 컨텍스트 길이에 대해서 성능이 좋음</td>
            <td>단방향 방식으로 자연어 이해 작업(NLU)에서 비교적 성능이 낮음</td>
        </tr>
        <tr>
            <td>인코더-디코더</td>
            <td>BART,T5</td>
            <td>생성과 이해 모두 뛰어남, 이해 작업에서 양방향 방식을 사용</td>
            <td>복잡함, 학습에 많은 컴퓨팅 자원 필요</td>
        </tr>
    </tbody>
</table>


##### 인코더를 활용한 BERT
- 양방향 문맥을 모두 활용해 텍스트를 이해한다.
- 입력 토큰의 일부를 마스크 토큰으로 대체하고 마스크 토큰을 맞추는 마스크 언어 모델링 과제를 통해 사전 학습
- 다운스트림 과제에 따라 미세 조정해 사용
- 텍스트 분류뿐만 아니라 토큰 분류, 질문 답변, 자연어 추론 등 다양한 자연어 이해 작업에서 훌륭한 성능
##### 디코더를 활용한 GPT
- 생성 작업의 경우 입력 토큰이나 이전까지 생성한 토큰만을 문맥으로 활용하는 **인과적 언어 모델링**을 사용하기 떄문에 단뱡향 방식
- 다음 토큰을 예측하는 방식으로 사전 학습 수행
##### 인코더와 디코더를 모두 사용하는 BART,T5
- BART는 모델 사전 학습을 위해 입력에 노이즈를 추가하고 노이즈가 제거된 결과를 생성하는 과제 수행
- 인코더 부분이 양방향 추론이 가능하다는 점이 BART의 특징이며 더 자유로운 변형 추가가 가능하다는 점에서 BERT와 차이가 있음
- T5는 입력의 시작에 과제 종류를 지정해 하나의 모델에서 다양한 동작을 하도록 학습시킨 점이 특징
- 

# 사전 학습 매커니즘
### 인과적 언어 모델링
- 문장의 시작부터 끝까지 순차적으로 단어를 예측하는 방식. 이전에 등장한 단어를 바탕으로 다음에 등장할 단어를 예측
- GPT 같은 생성 트랜스포머 모델에서는 인과적 언어 모델링을 핵심적인 학습 방법으로 사용
### 마스크 언어 모델링
- 입력 단어 일부를 마스크 처리하고 그 단어를 맞추는 작업으로 모델을 학습시킨다.
- 인과적 언어 모델링은 앞에서부터 뒤로 순차적으로 생성하는데, 이 방식은 지금까지 생성한 문맥만 활용할 수 있다는 한계가 있음.
- 
   